In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

<IPython.core.display.Javascript object>

In [3]:
from CoreConfig.CoreConfig import CoreConfig
from DBUtility.DBUtility import DBSQLiteConnection, CreateEventsTableIfNotExist, CreateNPSTableIfNotExist
from DBUtility.DBUtility import FetchEventsRecordWithinSamePerson, FetchEventsLatestRecordWithSamePerson, FetchEventsRecordAll, FetchNPSRecordAll
from DBUtility.DBUtility import InsertEventRecord, InsertNPSRecord
from pandas import DataFrame
import datetime


import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
from IPython.display import display_pretty
import pandas as pd
from ipywidgets import Layout, Button, Box
from StylingUtility.StylingUtility import CustomStyling

# CoreConfig.initialize_core_config()
# DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())
# CreateEventsTableIfNotExist().execute()


### Prototype - Update Record

In [4]:
from DBUtility.DBUtility import CreatePOCTableIfNotExist, InsertPOCRecord, FetchPOCRecordAll, DropTable, UpdatePOCRecord, GetPOCStatus

In [5]:
# get configuration details
CoreConfig.initialize_core_config()

# establish database connection
DBSQLiteConnection().configure_connection_information(**CoreConfig().get_sqlite3_db_connection_info())

# create table if not exist
CreatePOCTableIfNotExist().execute()

Initialize nab POC table failed. Please check your connection
table nab_poc_status already exists


In [ ]:
DropTable('nab_poc_status').execute()

In [11]:
InsertPOCRecord('POC_1', 'Old_Status').execute()
InsertPOCRecord('POC_2', 'Old_Status').execute()
InsertPOCRecord('POC_3', 'Old_Status').execute()
InsertPOCRecord('POC_4', 'Old_Status').execute()
InsertPOCRecord('POC_5', 'Old_Status').execute()

In [ ]:
UpdatePOCRecord('POC_1', 'New_Status').execute()

In [6]:
df_out_poc = widgets.Output(layout={'border': '1px solid black', 'aligh_items' : 'center'})

with df_out_poc:
    df = DataFrame(FetchPOCRecordAll().execute(), columns=['POC_Name', 'POC_Status'])
    display(df)


POC_dict = {}
POC_list = FetchPOCRecordAll().execute()
for name in [row[0] for row in POC_list]:
    POC_dict[name] = name
    
POC_Name = widgets.Dropdown(options= POC_dict,
                           value = 'POC_1',
                           description='Change status for:',
                           layout=CustomStyling.box_layout,
                           style=CustomStyling.style)

default_status = 'New_Status'
Status_Option = widgets.Dropdown(options= {'Old_Status': 'Old_Status',
                                     'New_Status': 'New_Status'},
                           value = default_status,
                           description='Update status to:',
                           layout=CustomStyling.box_layout,
                           style=CustomStyling.style)





button_to_update_form = widgets.Button(
                       description='Update Form',
                       disable=False,
                       button_style ='', # success
                       tooltip='Click me',
                       icon='check'
                       )

def on_update_form_button_clicked_poc(b):
    default_status = GetPOCStatus(POC_Name.value).execute()
    Status_Option.value = default_status


button_to_update_form.on_click(on_update_form_button_clicked_poc)







button_poc = widgets.Button(
           description='Update Status',
           disable=False,
           button_style ='', # success
           tooltip='Click me',
           icon='check'
           )

def on_submit_form_button_clicked_poc(b):
#     outputs = { 'POC_Name' : [POC_Name.value],
#                 'Status_Option' : [Status_Option.value]
#                }
    UpdatePOCRecord(POC_Name.value, Status_Option.value).execute()
    df = DataFrame(FetchPOCRecordAll().execute(), columns=['POC_Name', 'POC_Status'])
    df_out_poc.clear_output()
    with df_out_poc:
        display(df)
        
button_poc.on_click(on_submit_form_button_clicked_poc)





# user interface organised using VBox (vertical)
ui_poc = widgets.VBox(children=[df_out_poc,
                            POC_Name,
                            button_to_update_form,
                            Status_Option,
                            button_poc], layout=CustomStyling.items_layout)

# live display of values, for debugging purposes
def form_poc(POC_Name, Status_Option):
    #display(POC_Name, Status_Option)
    return POC_Name, Status_Option

out_poc = interactive_output(form_poc, {'POC_Name': POC_Name,
                                'Status_Option': Status_Option
                               })

display(ui_poc, out_poc)

A Jupyter Widget

A Jupyter Widget